In [4]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 47.0 MB/s eta 0:00:00


In [5]:
import mysql.connector
from mysql.connector import Error

hostname = "4kz9e.h.filess.io"
database = "olistproject_adjective"
port = "3307"
username = "olistproject_adjective"
password = "45069a5e486c8ad19dcfb1b713ab3a720ba108d8"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)

except Error as e:
    print("Error while connecting to MySQL or inseting data", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Connected to MySQL Server version  8.0.36-28
You're connected to database:  ('olistproject_adjective',)
MySQL connection is closed


In [6]:
import pandas as pd

In [9]:
data = pd.read_csv('olist_order_payments_dataset.csv')
data.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


Read csv and save it to sql table

In [10]:
import mysql.connector
from mysql.connector import Error

hostname = "4kz9e.h.filess.io"
database = "olistproject_adjective"
port = "3307"
username = "olistproject_adjective"
password = "45069a5e486c8ad19dcfb1b713ab3a720ba108d8"

csv_file_path = "olist_order_payments_dataset.csv"
table_name = "olist_order_payments"

try:
    connection = mysql.connector.connect(host=hostname, database=database, user=username, password=password, port=port)
    if connection.is_connected():
        print("Connected to MySQL Server Successfully")

        cursor = connection.cursor()
        cursor.execute(f"Drop table if exists {table_name}")
        print(f"Table {table_name} dropped because already existed")

        create_table_query = f"""
        CREATE TABLE {table_name} (
            order_id VARCHAR(255),
            payment_sequential INT,
            payment_type VARCHAR(255),
            payment_installments INT,
            payment_value FLOAT
        )
        """
        cursor.execute(create_table_query)
        print(f"Table {table_name} created successfully")

        data = pd.read_csv(csv_file_path)
        print("CSV data loaded to pandas dataframe")

        batch_size = 1000
        total_records = len(data)

        print(f"starting data insertion into {table_name} in batches of {batch_size}")

        for start in range(0, total_records, batch_size):
            end = start + batch_size
            batch = data.iloc[start:end]

            batch_records = [tuple(row) for row in batch.values]

            #insert_query
            insert_query = f"""
            INSERT INTO {table_name}
            (order_id, payment_sequential, payment_type, payment_installments, payment_value)
            VALUES (%s, %s, %s, %s, %s);
            """

            cursor.executemany(insert_query, batch_records)
            connection.commit()

            print(f"Batch {start//batch_size + 1} inserted successfully")

        print(f"All {total_records} records inserted successfully into {table_name}")

except Error as e:
    print("Error while connecting to MySQL or inseting data", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Connected to MySQL Server Successfully
Table olist_order_payments dropped because already existed
Table olist_order_payments created successfully
CSV data loaded to pandas dataframe
starting data insertion into olist_order_payments in batches of 1000
Batch 1 inserted successfully
Batch 2 inserted successfully
Batch 3 inserted successfully
Batch 4 inserted successfully
Batch 5 inserted successfully
Batch 6 inserted successfully
Batch 7 inserted successfully
Batch 8 inserted successfully
Batch 9 inserted successfully
Batch 10 inserted successfully
Batch 11 inserted successfully
Batch 12 inserted successfully
Batch 13 inserted successfully
Batch 14 inserted successfully
Batch 15 inserted successfully
Batch 16 inserted successfully
Batch 17 inserted successfully
Batch 18 inserted successfully
Batch 19 inserted successfully
Batch 20 inserted successfully
Batch 21 inserted successfully
Batch 22 inserted successfully
Batch 23 inserted successfully
Batch 24 inserted successfully
Batch 25 inser

In [1]:
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.6 MB/s eta 0:00:00


In [3]:
# importing module
from pymongo import MongoClient

hostname = "m1js9.h.filess.io"
database = "olistNoSql_establish"
port = "27018"
username = "olistNoSql_establish"
password = "34647078d307789d6a9c068c87eba23e02d18a59"

uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database

# Connect with the portnumber and host
client = MongoClient(uri)

# Access database
mydatabase = client[database]


In [4]:
# prompt: read product category name translation csv file and save it to above mongodb

import pandas as pd
from pymongo import MongoClient

# Assuming your CSV file is named 'product_category_name_translation.csv'
# and is in the same directory as your notebook.

try:
    # Read the CSV file into a pandas DataFrame
    df = pd.read_csv('product_category_name_translation.csv')

    # Convert the DataFrame to a list of dictionaries
    data_dict = df.to_dict('records')

    # MongoDB connection details (replace with your actual credentials)
    hostname = "m1js9.h.filess.io"
    database = "olistNoSql_establish"
    port = "27018"
    username = "olistNoSql_establish"
    password = "34647078d307789d6a9c068c87eba23e02d18a59"

    uri = "mongodb://" + username + ":" + password + "@" + hostname + ":" + port + "/" + database
    client = MongoClient(uri)
    db = client[database]

    # Specify the collection name
    collection_name = "product_categories"  # Choose a suitable name
    collection = db[collection_name]

    # Insert the data into the MongoDB collection
    collection.insert_many(data_dict)
    print(f"Successfully inserted data into collection '{collection_name}'")

except FileNotFoundError:
    print("Error: 'product_category_name_translation.csv' not found.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    # Close the MongoDB connection
    if client:
        client.close()


Successfully inserted data into collection 'product_categories'
